# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

It is a **classification problem**.
- The output is binary. It is a Yes-no answer to the question 'Does the student need early intervention?'.
- Thus **the output is discrete**.
- Regression deals with continuous output, whereas classification deals with discrete output.
- So this supervised learning problem is a classification problem, specifically one with **two classes**.

If we had a continuous score ranging from 0 to 1 that indicated the extent to which students might need early intervention, this could be a regression problem. But we don't have this data for students, so it's not a regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [54]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [65]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
# Don't count labels column
n_features = len(student_data.iloc[0]) -1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/n_students * 100

# Print the results
print("Total number of students （number of datapoints): {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed (graduates): {}".format(n_passed))
print("Number of students who failed (non-graduates): {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students （number of datapoints): 395
Number of features: 30
Number of students who passed (graduates): 265
Number of students who failed (non-graduates): 130
Graduation rate of the class: 67.09%


In [56]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


In [57]:
# Experiment to see if `failures` are a good predictor of `passed`

student_data[['failures', 'passed']]
pp, pf, fp, ff = 0, 0, 0, 0
for i in range(len(student_data)):
    if student_data.iloc[i]['failures'] > 0:
        if student_data.iloc[i]['passed'] == 'no':
            ff += 1
        else:
            fp += 1
    else:
        if student_data.iloc[i]['passed'] == 'no':
            pf += 1
        else:
            pp += 1
print("pp: ", pp, "pf: ", pf, "fp: ", fp, "ff: ", ff)

pp:  234 pf:  78 fp:  31 ff:  52


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [58]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [59]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [60]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_all, y_all = shuffle(X_all, y_all)

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, test_size=num_test)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

**Description of data:**
- 395 datapoints (few) -> should not use that many features if we want an accurate, generalisable model (Curse of Dimensionality)
- 30 features (non-trivial but not high compared to text learning applications that may have 50,000 features) 

**Model 1: Random Forests**
<table>
<th></th><th>Random Forests</th>
<tr><td>Application</td><td><ul><li>Predicting prices of futures (stocks) (<a href="http://www.scientific.net/AMM.740.947">The Application of Random Forest in Finance, 2015"</a>).</li></ul></td></tr>
<tr><td>Strengths</td><td><ul><li>Handles binary features well because it is an ensemble of decision trees.</li><li>Handle high dimensional spaces and large numbers of training examples well.</li><li>Does not expect linear features or features that interact linearly.</li></ul></td></tr>
<tr><td>Weaknesses</td><td><ul><li>May overfit especially for noisy training data</li></ul></td></tr>
<tr><td>Why it's a good candidate</td><td><ul><li>Handles binary features well -> We have constructed the dataset such that we have many binary features.
</li><li>It is often quite accurate.</li></ul></td></tr>
</table>


**Model 2: Naive Bayes (GaussianNB)**
<table>
<th></th><th>Naive Bayes</th>
<tr><td>Application</td><td><ul><li>Text learning.</li></ul></td></tr>
<tr><td>Strengths</td><td><ul><li>Computationally efficient.</li><li>Can deal with many features (and so is used in text learning where there may be 50,000 features).</li></ul></td></tr>
<tr><td>Weaknesses</td><td><ul><li>Independent features assumption is likely false here.</li><li>E.g. `Medu` may be associated with `Fedu` because couples often meet at university or at workplaces where they may have similar jobs.</li></ul></td></tr>
<tr><td>Why it's a good candidate</td><td><ul><li>Efficient -> Problem stated they care about computational cost.</li><li>Can deal with many features -> There are 30 features in our dataset.</li></ul></td></tr>
</table>
    

**Model 3: Logistic Regression**

<table>
<th></th><th>Logistic Regression</th>
<tr><td>Application</td><td><ul><li>Predict whether or not high school students might smoke cigarettes (<a href="http://www.aabri.com/manuscripts/10617.pdf">Multiple logistic regression analysis of cigarette use among
high school students, 2009</a>).</li></ul></td></tr>
<tr><td>Strengths</td><td><ul><li>Is simple and has low variance -> robust to noise and is less likely to over-fit.</li></ul></td></tr>
<tr><td>Weaknesses</td><td><ul><li>Assumes there is one smooth linear decision boundary (features are linearly separable).</li></ul></td></tr>
<tr><td>Why it's a good candidate</td><td><ul><li>Output is binary (which is what we want).</li><li>Efficient (we care about computational cost).</li><li>Output can be interpreted as a probability, so it may be useful in prioritising students for intervention later on.</li><li>Unlikely to overfit (Good to compare with Random Forests).</li></ul></td></tr>
</table>

Reference documents:
* [What are the advantages of different classification algorithms?](https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms)

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [61]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print("\n")

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [66]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state=0)
clf_B = GaussianNB()
clf_C = LogisticRegression(random_state=0)

# TODO: Set up the training set sizes
# Previously shuffled
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for j in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        train_predict(clf, j[0], j[1], X_test, y_test)

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0516 seconds
Score:  0.97
Made predictions in 0.0039 seconds.
F1 score for training set: 0.9774.
Score:  0.652631578947
Made predictions in 0.0024 seconds.
F1 score for test set: 0.7556.


Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0095 seconds
Score:  0.985
Made predictions in 0.0027 seconds.
F1 score for training set: 0.9889.
Score:  0.684210526316
Made predictions in 0.0018 seconds.
F1 score for test set: 0.7727.


Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0146 seconds
Score:  0.986666666667
Made predictions in 0.0065 seconds.
F1 score for training set: 0.9901.
Score:  0.642105263158
Made predictions in 0.0019 seconds.
F1 score for test set: 0.7344.


Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0018 seconds
Score:  0.82
Made predictions in 0.0008 seconds.
F1 score

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Random Forest**  

| Training Set Size | Training Time (s) | Prediction Time (s) (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0516                   |     0.0024                   |      0.9774            |         0.7556        |
| 200               |        0.0095          |            0.0018            |          0.9889        |       **0.7727**          |
| 300               |           0.0146              |         0.0019               |        0.9901          |    0.7344      |

* High F1 score for train (0.97-0.99) vs lower F1 score for test (0.73-0.77) indicates there is overfitting and that the model is not generalising well.
* F1 test score decreases as training set size increases, again suggesting that there is overfitting.
* Training time is high. (over 10x that of GaussianNB, KNeighborsClassifier)

** Classifer 2 - GaussianNB**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0018               |     0.0007                   |     0.8392             |     **0.7591**            |
| 200               |     0.0007                    |   0.0002                     |   0.8309               |     0.7424     |
| 300               |     0.0011             |    0.0003                    |    0.8099              |    0.7463             |

** Classifer 3 - Logistic Regression**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0027                    |        0.0001                |    0.8872              |   0.7328              |
| 200               |     0.0017             |          0.0002              |          0.8489        |          0.7612       |
| 300               |        0.0026                 |      0.0001                  |    0.8337              |     **0.7883**     |

It's doing surprisingly well.

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

I chose **Logistic Regression**.

1. **Performance (important)**: Logistic Regression had the **highest F1 score**. 
    * F1 score is a combined measure of (the harmonic mean of) precision and recall.
        - Precision is X and 
        - Recall is Y.
    * The other comparable model is SVC. Logistic Regression and SVC have maximum F1 scores of **0.7883** and 0.7808 respectively, with both attaining F1 max at 300 training points. They are significantly higher than the third highest F1 score at 0.7591 for GaussianNB.

2. **Cost** (measured by training and prediction times):
    * Out of the two models with similar F1 score, Logistic Regression has the **shorter training and prediction time** at < 50% that af SVC's time (**0.0026s to train, 0.0001s to predict** vs SVC 0.0053s to train and 0.0013 to predict). 
    * The training time is not too high and the prediction time is extremely low at 0.0001s.
    * Since minimising computational cost is a concern, Logistic Regression seems like a good choice.

3. **Available data**
    * This is taken into account by the F1 scores used to assess the models. Three models - KNeighborsClassifier, SVC and Logistic Regression - have F1 scores that increase as the number of training datapoints increases. These models may perform even better if we had, say, 500 training data points. One of SVC or KNeighborsClassifier might outperform Logistic Regression. But we only have 300 data points, so we need to make choices based on the F1 scores in the tables.

**Note 1: Backup in case even Logistic Regression is too computationally expensive: GaussianNB**

If the computational cost is considered too great, we choose KNeighborsClassifier, which has a training time of 0.0009s, a prediction time of 0.0004s and an F1 score of 0.7591 on 100 training points, the third highest F1 score and the shortest (training time + prediction time).
* It is only trained on 100 datapoints which makes it feel a bit dodgy. I considered choosing KNeighborsClassifier which has the fourth highest F1 score (max at 300 training points) and a shorter training time, but its prediction time of 0.0019s is quite high, making computational cost high.

**Note 2: This may not be the optimal model because we did not tune any parameters.**
* The default parameters for e.g. Decision Trees may just be really bad for this example.
* If we wanted to choose the best model, we should compare versions of the models with tuned parameters.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

**The model is trained by** trying to fit an S-shaped curve to our data. This curve maps input features to the whether or not a student passed. The model picks this curve by minimising some error between the curve and our training data. This curve is expressed in the form of an equation (which becomes our model for prediction).

**The model makes a prediction** putting the input features (have they failed before? Do they have access to Internet at home?) into an equation we got during training. The equation gives us an output which tells us the chance of this student passing. If we predict that the student is more likely to pass than to fail, they are classified as 'passed' and vice versa.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [64]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    score = clf.score(features, target.values)
    end = time()
    print("Score: ", score)
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


# TODO: Create the parameters list you wish to tune
parameters = { "penalty":["l2","l1"], 
              # "tol":[0.00001, 0.0001, 0.001, 0.1, 1], 
               "C":[1,10,100,1000],
              }

# TODO: Initialize the classifier
clf = LogisticRegression()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
print(grid_obj)
# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
print("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l2', 'l1'], 'C': [1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Score:  0.77
Made predictions in 0.0008 seconds.
Tuned model has a training F1 score of 0.8442.
Score:  0.621052631579
Made predictions in 0.0008 seconds.
Tuned model has a testing F1 score o

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

<table>
<th>Attempt</th><th>F1 train score</th><th>F1 test score</th>
<tr><td>1 (allow "penalty" and "C" to vary)</td><td>0.8288</td><td>0.7801</td></tr>
<tr><td>2 (only allow "C" to vary)</td><td>0.8337</td><td>0.7883</td></tr>
<tr><td>0 (untuned model)</td><td>0.8337</td><td>0.7883</td></tr>
</table>

- The train and test scores are both lower than the untuned version if I allow both "penalty" and "C" to vary.
- If I only allow "C" to vary, GridSearchCV chooses "C"=1, which is the same as that of the untuned model. The train and test scores are thus the same as the untuned version.
- Why would GridSearchCV pick `penalty="l1"` if `penalty="l2"` produces better training F1 scores (all other factors held constant)?
    - One hypothesis was that it might be maximising another metric. This was not the case. I also printed the accuracy score and found that when I allowed GridSearchCV to choose the "penalty" and "C" values (as opposed to only "C"), the accuracy score was lower.


Note: We need to be wary of optimising for the testing set's F1 score. The test set is supposed to be a stand-in for future cases (generalising).



> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.